In [1]:
import pandas as pd
df = pd.read_csv("school_shooting.csv")
df['Date'] = pd.to_datetime(df['Date'])

df.tail()


,Date,Death,Injuries,Location,State,School_Name,School_Type,Latitude,Longitude
332,2013-03-21,0,0,Southgate,MI,Davidson Middle School,Middle School,42.206116,-83.204609
333,2013-02-27,0,0,Atlanta,GA,Henry W. Grady High School,High School,33.749099,-84.390185
334,2013-02-13,0,0,San Leandro,CA,Hillside Elementary School,Elementary School,37.724930,-122.156077
335,2013-02-01,0,0,Altanta,GA,Morehouse College,College or University,33.749000,-84.388000
336,2013-01-08,0,0,Fort Myers,FL,Apostolic Revival Center Christian School,Elementary School,26.640628,-81.872308


In [2]:
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float
Base = declarative_base()

engine = create_engine("sqlite:///school.sqlite")
conn = engine.connect()

In [3]:
class school_shoot(Base):
    __tablename__ = "School_shooting"
    id = Column(Integer, primary_key=True)
    Date = Column(String) # for late using the date column, we need to use pandas to convert it back to datetime format for analysis 
    Death = Column(Integer)
    Injuries = Column(Integer)
    Location= Column(String)
    State = Column(String)
    School_Name = Column(String)
    School_Type = Column(String)
    Latitude= Column(Float)
    Longitude = Column(Float)
    
    def __repr__(self):
        return f"id={self.id}, Date  = {self.Date}, Death ={self.Death}, Injuries = {self.Injuries}, State = {self.State},Location = {self.Location}, School_Name = {self.School_Name}, School_Type = {self.School_Type}, Latitude = {self.Latitude}, Longitude = {self.Longitude} "

In [4]:
Base.metadata.create_all(engine)

In [5]:
df = pd.read_csv("school_shooting.csv")

In [6]:
school_dict = df.to_dict(orient = 'records')
metadata = MetaData(bind=engine)
metadata.reflect()
table1 = sqlalchemy.Table('School_shooting', metadata, autoload=True)
conn.execute(table1.insert(), school_dict)


In [7]:
conn.execute("select * from School_shooting").fetchall()

[(1, '1/7/13', 1, 0, 'Fort Myers', ' FL', 'Apostolic Revival Center Christian School', 'Preschool', 26.640628000000003, -81.87230840000001),
 (2, '1/10/13', 0, 2, 'Taft', ' CA', 'Taft Union High School', 'High School', 50.9954169, -118.6467639),
 (3, '1/12/13', 0, 1, 'Detroit', ' MI', 'Osborn High School ', 'High School', 42.3486635, -83.0567375),
 (4, '1/15/13', 0, 2, 'St. Louis', ' MO', 'Stevens Institute of Business & Arts', 'College or University', 38.62727329999999, -90.19788890000001),
 (5, '1/15/13', 3, 0, 'Hazard', ' KY', 'Hazard Community and Technical College', 'College or University', 37.24954, -83.19322840000001),
 (6, '1/16/13', 1, 0, 'Chicago', ' IL', 'Chicago State University', 'College or University', 41.8755546, -87.6244212),
 (7, '1/22/13', 0, 3, 'Houston', ' TX', 'Lone Star College North Harris Campus', 'College or University', 29.7589382, -95.36769740000001),
 (8, '1/31/13', 0, 0, 'Phoenix', 'AZ', 'Cesar Chavez High School', 'High School', 33.448586600000006, -112.0

In [21]:
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func

In [22]:
engine = create_engine("sqlite:///school.sqlite")
Base = automap_base()
Base.prepare(engine, reflect= True)
Base.classes.keys()

['School_shooting']

In [23]:
session = Session(engine)
School_shooting = Base.classes.School_shooting

In [24]:
session.query(School_shooting.Date).all

<bound method Query.all of <sqlalchemy.orm.query.Query object at 0x11286b160>>

### need Lat, long info for plotting, and hover tick for city name, state name, school name, death, injury

In [25]:
import pandas as pd
import numpy as np

In [26]:
def getData():
    result = session.query(School_shooting.Date, School_shooting.Location, School_shooting.State, School_shooting.Latitude, 
                       School_shooting.Longitude, School_shooting.School_Name, School_shooting.Death,
                      School_shooting.Injuries, School_shooting.School_Type).all()
    df2 = pd.DataFrame(result)
    
    df2['Year'] = ['20' + str(j).split("/")[2] for j in df2['Date']] 
    return df2
df2 = getData()
df2.head()

,Date,Location,State,Latitude,Longitude,School_Name,Death,Injuries,School_Type,Year
0,1/7/13,Fort Myers,FL,26.640628,-81.872308,Apostolic Revival Center Christian School,1,0,Preschool,2013
1,1/10/13,Taft,CA,50.995417,-118.646764,Taft Union High School,0,2,High School,2013
2,1/12/13,Detroit,MI,42.348664,-83.056737,Osborn High School,0,1,High School,2013
3,1/15/13,St. Louis,MO,38.627273,-90.197889,Stevens Institute of Business & Arts,0,2,College or University,2013
4,1/15/13,Hazard,KY,37.249540,-83.193228,Hazard Community and Technical College,3,0,College or University,2013


In [14]:
type(np.asscalar(df2["Death"][0]))

int

In [15]:
df2['Date'] = pd.to_datetime(df2['Date'])
mask = df2["Year"] == str(2014)
yr = df2[mask]
yr['Date'] = [str(j).split(" ")[0] for j in yr['Date']]
df = yr
df
names = list(df.columns)
Outerlst = []
for i in range(len(df)):
    innerList = []
    for j in range(len(names)):
        innerList.append(df.iloc[i, names.index(names[j])])
        #print(innerList)
    Outerlst.append(dict(zip(names, innerList)))
for index in range(len(Outerlst)):
    
    for key, value in Outerlst[index].items():
        
        if value == str:
            next
        else:
            Outerlst[index][key] = np.asscalar(np.array([value])) 
            


/Users/apple/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [16]:
def map(year):
    
    df2['Date'] = pd.to_datetime(df2['Date'])
    mask = df2["Year"] == str(year)
    yr = df2[mask]
    yr['Date'] = [str(j).split(" ")[0] for j in yr['Date']]
    df = yr
    df
    names = list(df.columns)
    Outerlst = []
    for i in range(len(df)):
        innerList = []
        for j in range(len(names)):
            innerList.append(df.iloc[i, names.index(names[j])])
            #print(innerList)
        Outerlst.append(dict(zip(names, innerList)))
    for index in range(len(Outerlst)):

        for key, value in Outerlst[index].items():

            if value == str:
                next
            else:
                Outerlst[index][key] = np.asscalar(np.array([value])) 

    
    return Outerlst

In [17]:
type(map(2014)[0]['Latitude'])

/Users/apple/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


float

## get year data

In [18]:
def yearOPtion():
    df2= getData()
    yearList = []
    str_date = [str(date) for date in df2['Date'].dt.year]
    
    yearlst = list(set(str_date))
    yearlst.sort()
    return jsonify(yearlst)

In [19]:
# get states

yearList = []
for state in df2['State']:
    
    yearList.append(state)

In [20]:
for title in list(yr13.columns):
    littlelist.append([i for i in yr13[title]])

NameError: name 'yr13' is not defined

### getting bar graph data

In [27]:
def getData():
    result = session.query(School_shooting.Date, School_shooting.Location, School_shooting.State, School_shooting.Latitude, 
                       School_shooting.Longitude, School_shooting.School_Name, School_shooting.Death,
                      School_shooting.Injuries, School_shooting.School_Type).all()
    df2 = pd.DataFrame(result)
    df2["Date"] = pd.to_datetime(df2["Date"])
    df2['Year'] = ['20' + str(j).split("-")[2] for j in df2['Date']] 
    df2['State'] = [i.strip() for i in df2['State']]
    return df2
df2 = getData()
df2 = df2.dropna()
df2.head()

,Date,Location,State,Latitude,Longitude,School_Name,Death,Injuries,School_Type,Year
0,2013-01-07,Fort Myers,FL,26.640628,-81.872308,Apostolic Revival Center Christian School,1,0,Preschool,2007 00:00:00
1,2013-01-10,Taft,CA,50.995417,-118.646764,Taft Union High School,0,2,High School,2010 00:00:00
2,2013-01-12,Detroit,MI,42.348664,-83.056737,Osborn High School,0,1,High School,2012 00:00:00
3,2013-01-15,St. Louis,MO,38.627273,-90.197889,Stevens Institute of Business & Arts,0,2,College or University,2015 00:00:00
4,2013-01-15,Hazard,KY,37.249540,-83.193228,Hazard Community and Technical College,3,0,College or University,2015 00:00:00


In [ ]:
df2.groupby(["State",'School_Type']).agg({"School_Name": "count"})

In [ ]:
df2['State'].unique()

In [ ]:
mask = df2['State'] == "PA"
df = df2[mask]
df
count = df.groupby(['School_Type']).agg({"School_Name": "count"})
count = count.reset_index()
df

In [ ]:
df2['School_Type'].unique()

In [ ]:
schoolType =list(df['School_Type'].unique())
schoolType

In [ ]:
School_list = []
for types in schoolType:
    mask = df["School_Type"] == types
    School_list.append(list(df[mask]['School_Name']))
School_list

In [ ]:
ticks = dict(zip(list(schoolType), School_list))
ticks

In [28]:
def schoolType(name):
    result = session.query(School_shooting.State, School_shooting.School_Name, School_shooting.School_Type).all()
    df2 = pd.DataFrame(result)
    df2['State'] = [i.strip() for i in df2['State']]
    mask = df2['State'] == name
    df = df2[mask]
    count = df.groupby(['School_Type']).agg({"School_Name": "count"})
    count = count.reset_index()
    schoolType =list(df['School_Type'].unique())
    School_list = []
    for types in schoolType:
        mask = df["School_Type"] == types
        School_list.append(list(df[mask]['School_Name']))
        
    ticks = dict(zip(list(schoolType), School_list))
    return ticks

In [ ]:
schoolType('MN')


In [ ]:
len(dictt['College or University'])

In [ ]:
def State_schoolType():
    df2['State'] = [i.strip() for i in df2['State']]    
    statelst = list(df2['State'].unique())
    statelst.sort()
    stateDict = {}
    for name in statelst:
        stateDict[name] = schoolType(name)
        
    return stateDict 

In [ ]:
State_schoolType()

In [ ]:
stateDict = {}
for name in statelst:
    stateDict[name] = schoolType(name)

In [ ]:
stateDict

In [ ]:
def schoolType(name):
        result = session.query(School_shooting.State, School_shooting.School_Name, School_shooting.School_Type).all()
        df2 = pd.DataFrame(result)
        df2 = df2.dropna()
        df2['State'] = [i.strip() for i in df2['State']]
        mask = df2['State'] == name
        df = df2[mask]
        count = df.groupby(['School_Type']).agg({"School_Name": "count"})
        count = count.reset_index()
        schoolType =list(df['School_Type'].unique())
        School_list = []
        for types in schoolType:
            mask = df["School_Type"] == types
            School_list.append(list(df[mask]['School_Name']))
            
        ticks = dict(zip(schoolType, School_list))
        return ticks

In [ ]:
def State_schoolType(stateName):
    
    result = session.query(School_shooting.State, School_shooting.School_Name, School_shooting.School_Type).all()
    df2 = pd.DataFrame(result)
    df2 = df2.dropna()
    df2['State'] = [i.strip() for i in df2['State']]    
    statelst = list(df2['State'].unique())
    statelst.sort()
    stateDict = {}
    for name in statelst:
        stateDict[name] = schoolType(name)
    

    return stateDict[f"{stateName}"]


In [ ]:
State_schoolType(PA)

In [ ]:
name = 'PA'
print(f"{name}")

## table 1 end point

In [29]:
import pandas as pd
def dictTranform(dict_list):
    df = pd.DataFrame(dict_list)
    names = list(df.columns)
    Outerlst = []
    for i in range(len(df)):
        innerList = []
        for j in range(len(names)):
            innerList.append(df.loc[i, names[j]])
        Outerlst.append(dict(zip(names, innerList)))
    return Outerlst

In [30]:
def schoolType(name):
        result = session.query(School_shooting.State, School_shooting.School_Name, School_shooting.School_Type).all()
        df2 = pd.DataFrame(result)
        df2 = df2.dropna()
        df2['State'] = [i.strip() for i in df2['State']]
        mask = df2['State'] == name
        df = df2[mask]
        count = df.groupby(['School_Type']).agg({"School_Name": "count"})
        count = count.reset_index()
        schoolType =list(df['School_Type'].unique())
        School_list = []
        for types in schoolType:
            mask = df["School_Type"] == types
            School_list.append(list(df[mask]['School_Name']))
            
        ticks = dict(zip(schoolType, School_list))
        return ticks


In [31]:
result = session.query(School_shooting.State, School_shooting.School_Name, School_shooting.School_Type).all()
df2 = pd.DataFrame(result)
df2 = df2.dropna()
df2['State'] = [i.strip() for i in df2['State']]    
statelst = list(df2['State'].unique())
statelst.sort()
stateDict = {}
for name in statelst:
    stateDict[name] = schoolType(name)
target = stateDict['FL']



In [32]:
keylength = []
length = []
for key, val in target.items():
    keylength.append(key) 
    length.append(len(val))
length.index(max(length))
for key, val in target.items():
    if len(val) != max(length):
        target[key] = target[key] + ["-" for i in range(max(length)-len(val))]

In [33]:
final = dictTranform(target)
final

[{'Preschool': 'Apostolic Revival Center Christian School',
  'College or University': 'University of Central Florida',
  'Elementary School': 'Agape Christian Academy',
  'High School': 'West Orange High School',
  'K-12 School': 'North Broward Preparatory School'},
 {'Preschool': 'The Academy of Knowledge Preschool',
  'College or University': 'Eastern Florida State College',
  'Elementary School': 'Franklin County Elementary School',
  'High School': 'Miami Carol City High School',
  'K-12 School': '-'},
 {'Preschool': 'Everglades City School',
  'College or University': 'Florida State University',
  'Elementary School': 'Oscar Patterson Elementary School',
  'High School': 'Vanguard High School',
  'K-12 School': '-'},
 {'Preschool': '-',
  'College or University': 'Bethune-Cookman University',
  'Elementary School': 'Rock Crusher Elementary School',
  'High School': 'Marjory Stoneman Douglas High School',
  'K-12 School': '-'},
 {'Preschool': '-',
  'College or University': 'Unive

In [34]:
target

{'Preschool': ['Apostolic Revival Center Christian School',
  'The Academy of Knowledge Preschool',
  'Everglades City School',
  '-',
  '-',
  '-',
  '-',
  '-'],
 'College or University': ['University of Central Florida',
  'Eastern Florida State College',
  'Florida State University',
  'Bethune-Cookman University',
  'University of West Florida',
  'University of South Florida',
  'University of Florida',
  'University of Miami'],
 'Elementary School': ['Agape Christian Academy',
  'Franklin County Elementary School',
  'Oscar Patterson Elementary School',
  'Rock Crusher Elementary School',
  'Greenwood Lakes Middle School',
  'Apostolic Revival Center Christian School',
  '-',
  '-'],
 'High School': ['West Orange High School',
  'Miami Carol City High School',
  'Vanguard High School',
  'Marjory Stoneman Douglas High School',
  'Forest High School',
  'Lecanto High School',
  'Alexander W. Dreyfoos School of the Arts',
  '-'],
 'K-12 School': ['North Broward Preparatory School'

# endpooint for table

In [ ]:
import pandas as pd
def getData():
    result = session.query(School_shooting.Date, School_shooting.Location, School_shooting.State, School_shooting.Latitude, 
                       School_shooting.Longitude, School_shooting.School_Name, School_shooting.Death,
                      School_shooting.Injuries, School_shooting.School_Type).all()
    df2 = pd.DataFrame(result)
    df2['Date'] = pd.to_datetime(df2['Date'])
    df2['Date'] = [str(i).split(' ')[0] for i in df2['Date']]
    comboLst = list(zip(df2['Death'][130:], df2["Injuries"][130:]))
    Death = []
    Injury = []
    for j in comboLst:
        if j[0] == 0 and j[1] == 0:
            Death.append("-")
            Injury.append("-")
        else:
            Death.append(j[0])
            Injury.append(j[1])
    df2['Death'] = list(df2['Death'][0:130]) + Death
    df2['State'] = [i.split()[0] for i in df2['State']]
    df2['Injuries'] = list(df2['Injuries'][0:130]) + Injury
    df2['Location'] = [i.upper() for i in df2['Location']] 
    Stype = []
    for i in df2['School_Type']:
        if i == None :
            Stype.append("-")
        else:
            Stype.append(i.upper().strip())

    df2['School_Type'] = Stype
    name = []
    for i in df2['School_Name']:
        if i == None :
            name.append("-")
        else:
            name.append(i.upper().strip())

    df2['School_Name'] = name
    
    nulllist = []
    for i in df2['School_Name']:
        if i == None:
            nulllist.append("-")
        else:
            nulllist.append(i)
    df2['School_Name'] = nulllist
    names = list(df2.columns)
    Outerlst = []
    for i in range(len(df2)):
        innerList = []
        for j in range(len(names)):
            innerList.append(df2.loc[i, names[j]])
        Outerlst.append(dict(zip(names, innerList)))
    
    return Outerlst

In [ ]:
getData()

In [ ]:
df2['School_Type'].isnull().unique()

In [ ]:
name = []
for i in df2['School_Name']:
    if i == None :
        name.append("-")
    else:
        name.append(i.upper().strip())
    
df2['School_Name'] = name

In [ ]:
df2['School_Name']

In [ ]:
str(type(None))

In [ ]:
for i in df2['School_Name']:
    if i is not None:
        print(i)
